# My model vectorizes the text and then then concatenates it with other features and then passing it to a Lightgbm classifier

Importing the libraries and dataset

In [2]:
import pandas as pd
import numpy as np
import string
import nltk

import matplotlib.pyplot as plt

import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import accuracy_score
import re

%matplotlib inline

In [3]:
df=pd.read_csv('train_data.csv')

In [37]:
df.head()

,index,User,Tweet,Retweet count,Likes count,Tweet value
0,0,1,"""Particular attention will be paid to understa...",21,40,2440.26
1,3,0,Coronavirus is like climate change on a three ...,1397,11017,4002.26
2,4,0,"Dear @SkyNews, can you explain this? Watch the...",1454,3877,21.32
3,5,1,@DrTedros It was the unanimous view of the Eme...,82,92,2684.28
4,10,1,"@DrTedros @_AfricanUnion @NCDCgov ""20% of pati...",414,577,2440.26


Preprocessing:
1)removing stopwords
2)removing url and other noisy data
3)removing emojis
4)removing punctuation

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [6]:
stop=stopwords.words('english')

In [7]:
text=df['Tweet']

In [8]:
def remove_stopwords(text):
        if text is not None:
            tokens = [x for x in word_tokenize(text) if x not in stop]
            return " ".join(tokens)
        else:
            return None

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)


def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


def remove_punct(text):
    exclude = set(string.punctuation)
    s = ''.join(ch for ch in text if ch not in exclude)
    return s

def clean_df(text):
    
    
    text = text.apply(lambda x : x.lower())
    
    text=text.apply(lambda x: remove_emoji(x))
        
    text=text.apply(lambda x : remove_URL(x))
        
    text=text.apply(lambda x : remove_html(x))
        
    text=text.apply(lambda x : remove_stopwords(x)) 
    
    text=text.apply(lambda x : remove_punct(x))
    
    text=text.replace('\s+', ' ', regex=True)
    
    return text

In [9]:
text=clean_df(text)

Vectorizing the Text using a vectorizer from scikitlearn

In [13]:
vectorizer = TfidfVectorizer(input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=5,
    max_features=None,
    vocabulary=None,
    binary=False,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,)
text = vectorizer.fit_transform(text)

In [15]:
text=pd.DataFrame.sparse.from_spmatrix(text)#converting the vectorized text into an dataframe object

In [16]:
text_matrix=pd.concat([df[['Retweet count', 'Likes count',
       'Tweet value']],text],axis=1)

In [17]:
text_matrix.head()

,Retweet count,Likes count,Tweet value,0,1,2,3,4,5,6,...,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391
0,21,40,2440.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1397,11017,4002.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1454,3877,21.32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,82,92,2684.28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,414,577,2440.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
x = text_matrix
y = df['User']

Preparing and selecting the best params and fitting on the model and then predicting the test data

In [47]:
import lightgbm
from sklearn.model_selection import GridSearchCV

In [49]:
model=lightgbm.LGBMClassifier()
kfold=KFold(n_splits=3,shuffle=True)

In [55]:
param={'n_estimators':[50,100,200,500],'learning_rate':[0.1,0.01,0.001],'num_leaves':[31,63,41]}

In [58]:
cross_val=GridSearchCV(model,param,cv=kfold,scoring='accuracy')

In [59]:
cross_val.fit(text_matrix,y,)

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
             error_score=nan,
             estimator=LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                      colsample_bytree=1.0,
                                      importance_type='split',
                                      learning_rate=0.1, max_depth=-1,
                                      min_child_samples=20,
                                      min_child_weight=0.001,
                                      min_split_gain=0.0, n_estimators=100,
                                      n_jobs=-1, num_leaves=31, objective=None,
                                      random_state=None, reg_alpha=0.0,
                                      reg_lambda=0.0, silent=True,
                                      subsample=1.0, subsample_for_bin=200000,
                                      subsample_freq=0),
             iid='deprecated', n_jobs=None,
             param_grid={'learni

In [60]:
cross_val.best_params_

{'learning_rate': 0.1, 'n_estimators': 500, 'num_leaves': 41}

In [61]:
model=cross_val.best_estimator_

In [62]:
predictions_train = model.predict(text_matrix)

In [63]:
print('Train Accuracy:', accuracy_score(y,predictions_train))

Train Accuracy: 1.0


 preparing and predicting test data:)

In [74]:
test=pd.read_csv('test_data.csv')

In [75]:
test_text=clean_df(test['Tweet'])

In [76]:
test_text=vectorizer.transform(test_text)

In [77]:
test_text=pd.DataFrame.sparse.from_spmatrix(test_text)

In [78]:
test_matrix=pd.concat([test[['Retweet count', 'Likes count',
       'Tweet value']],test_text],axis=1)

In [79]:
result=pd.DataFrame(model.predict(test_matrix))

In [80]:
index=pd.DataFrame(test['index'])

In [81]:
result=pd.concat([index,result],axis=1)
result.columns=['index','User']

In [82]:
result.to_csv('submission.csv',index=False)